In [5]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue
import pyaudio
import matplotlib.pyplot as plt
import soundfile as sf

p = pyaudio.PyAudio()
message = Queue()
recording = Queue()

CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2


# Create a button
record_button = widgets.Button(
    description="Record",
    disabled = False,
    button_style='info', 
    icon  = 'microphone'
)

stop_button = widgets.Button(
    description="Stop",
    disabled = False,
    button_style='warning',
    icon  = 'stop'
)

output = widgets.Output()

In [ ]:
def plot_audio(data):
    # Load the original audio waveform from a file
    audio_data = sf.read(data)

    # Plot the original audio waveform
    plt.subplot(2, 1, 1)
    plt.plot(audio_data)
    plt.title('Original Audio')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')


In [6]:
def record_audio(chunk = 1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input = True,
                    input_device_index = 1,
                    frames_per_buffer=chunk)
    
    global frames
    frames = []

    while not message.empty():
        data = stream.read(chunk)
        frames.append(data)
        frames.append(data)

        if len(frames) > (FRAME_RATE / chunk) * RECORD_SECONDS:
            recording.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [7]:
def start_record(data):
    message.put(True)
    with output:
        print("Recording...")
        record = Thread(target=record_audio)
        record.start()

        # transcribe = Thread(target=speech_recognition, args=(output,))
        # transcribe.start()

def plot_audio(data):
    print("Plotting...")
    plt.plot(data)
    plt.show()

def stop_record(data):
    with output:
        message.get()
        display("Stop recording")

    plot_audio(frames)




In [8]:
record_button.on_click(start_record)
stop_button.on_click(stop_record)

display(record_button, stop_button, output)

Button(button_style='info', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()